In [1]:
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ReadHDFSData") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .getOrCreate()

25/05/16 23:37:09 WARN Utils: Your hostname, mar3oo-Lenovo-ideapad-320-15IKB resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface wlp3s0)
25/05/16 23:37:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/16 23:37:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("hdfs://localhost:9000/user/hadoop/energy_data/iot_data.csv", header=True)
df.show()

25/05/16 23:37:25 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
                                                                                

+---------+--------+--------------------+-----------------------+
|    LCLid|stdorToU|            DateTime|KWH/hh (per half hour) |
+---------+--------+--------------------+-----------------------+
|MAC000002|     Std|2012-10-12 00:30:...|                     0 |
|MAC000002|     Std|2012-10-12 01:00:...|                     0 |
|MAC000002|     Std|2012-10-12 01:30:...|                     0 |
|MAC000002|     Std|2012-10-12 02:00:...|                     0 |
|MAC000002|     Std|2012-10-12 02:30:...|                     0 |
|MAC000002|     Std|2012-10-12 03:00:...|                     0 |
|MAC000002|     Std|2012-10-12 03:30:...|                     0 |
|MAC000002|     Std|2012-10-12 04:00:...|                     0 |
|MAC000002|     Std|2012-10-12 04:30:...|                     0 |
|MAC000002|     Std|2012-10-12 05:00:...|                     0 |
|MAC000002|     Std|2012-10-12 05:30:...|                     0 |
|MAC000002|     Std|2012-10-12 06:00:...|                     0 |
|MAC000002

In [4]:
df = df.withColumnRenamed("KWH/hh (per half hour) ", "KWH/hh")

In [5]:
# Cast the column to float
df = df.withColumn("KWH/hh", col("KWH/hh").cast("double"))

In [6]:
df = df.withColumn("datetime", to_timestamp("datetime", "yyyy-MM-dd HH:mm:ss"))

In [8]:
'''
# Sample 5% of data, stratified by household and tariff type
sample_df = df.sampleBy("LCLid", fractions={h: 0.0003 for h in df.select("LCLid").distinct().rdd.flatMap(lambda x: x).collect()})\
            .sampleBy("stdorToU", fractions={"Std": 0.0003, "ToU": 0.0003})

# Ensure we get at least 1 record per household
min_sample = df.groupBy("LCLid").count().withColumn("sample", col("count")*0.0003).filter(col("sample") < 1).select("LCLid")
sample_df = sample_df.union(df.join(min_sample, "LCLid").sample(0.0003))'''

In [10]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [18]:
# Register DataFrame as a temp view
df.createOrReplaceTempView("energy_data")

# Sample 0.0003 using SQL RAND() - avoids RDD operations
sample_df = spark.sql("""
  SELECT * FROM energy_data 
  WHERE RAND() <= 0.0003  -- 0.03% sample
""")

# Verify sample size
print(f"Sampled {sample_df.count():,} rows (original: {df.count():,})")

[Stage 29:=======================================================>(63 + 1) / 64]

Sampled 50,608 rows (original: 167,932,474)


In [19]:
sample_df

DataFrame[LCLid: string, stdorToU: string, datetime: timestamp, KWH/hh: double]

In [22]:
sample_pd_df = sample_df.toPandas()

In [23]:
sample_pd_df

,LCLid,stdorToU,datetime,KWH/hh
0,MAC000002,Std,2013-04-18 02:30:00,0.179
1,MAC000002,Std,2013-04-30 03:30:00,0.085
2,MAC000002,Std,2013-07-03 04:30:00,0.103
3,MAC000002,Std,2013-07-04 12:00:00,0.266
4,MAC000002,Std,2013-08-09 12:00:00,0.147
...,...,...,...,...
50603,MAC005564,ToU,2013-07-10 23:30:00,0.121
50604,MAC005564,ToU,2013-12-11 17:30:00,0.057
50605,MAC005564,ToU,2013-12-16 04:30:00,0.045
50606,MAC005564,ToU,2014-01-18 07:00:00,0.034


In [24]:
sample_pd_df.to_csv("./energy_consumption_sample.csv", index=False)